# The Science Bit

Preparing the data from https://data.gov.ie/ prior to doing the analyis.

In [2]:
import pandas as pd
import glob
import json
import pickle

## Fixers

There was some inconsistency in the way the data is recorded, between different elections and different local authorities. We may be able to realise the difference between `Non-Transferrable` and `Non_Transferrable` votes, but to the computer they are furballs. Therefore, I created three `.json` objects to ensure consistency across all streams.

In [3]:
with open("column_renamers/count.json") as f: count_columns = json.load(f)
with open("column_renamers/candidate.json") as f: candidate_columns = json.load(f)
with open("column_renamers/constituency-fixer.json") as f: con_fixer = json.load(f)

## Functions

Functions aren't as common in Jupyter Notebooks as they are in *regular* programming, but the data was sufficiently complex here to make them worthwhile. As such, I wrote five functions - three to handle the preparation of the main data structures, which are count, candidate and constituency records, and two helper functions used by the bigger functions - `name_candidates()` to give the candidates a consistent name, and `create_keys()`, to allow me to connect the data more easily.

In [4]:
def name_candidates(someDf):
    """
    A helper function to provide each candidate with a single name field, identifying them
    by full name and by party.
    """
    names = []
    for a, b in someDf.iterrows():
        name = " ".join([b['First Name'], b['Surname']])
        name = ", ".join([name, b['Party Abbreviation']])
        names.append(name)
        
    return names

In [5]:
def create_keys(someDf, election):
    """
    A helper function to provide each row of a database with a unique identifier, based on
    Constitution Number, Candidate Id and an ID for a particular election.
    """
    keys = []
    for a, b in someDf.iterrows():
        keys.append((election, b['Constituency Number'], b['Candidate Id']))
    return keys

In [6]:
def create_constituency_data(url):
    """
    read a constituency file and return it as a dictionary, keyed by
    Constituency Number.
    """
    df = pd.read_csv(url)
    df.index = df['Constituency Number']
    df['Constituency Name'] = df['Constituency Name'].map(con_fixer)
    del(df['Constituency Number'])
    
    return df.to_dict('index')

In [7]:
def create_candidate_data(url, election, constituencyDict):
    """
    read a candidate file and parse it.
    """
    df = pd.read_csv(url)
    df.rename(columns=candidate_columns, inplace=True)
    df['name'] = name_candidates(df)
    df['key'] = create_keys(df, election)
    df['seats'] = df['Constituency Number'].apply(lambda x: constituencyDict[x]['Seats Filled'])
    df['Result'] = df['Result'].apply(lambda x: x if x=="Elected" else "Not Elected")
    df['Constituency'] = df['Constituency'].map(con_fixer)
    df = df[['Constituency', 'Gender', 'Party', 'Count Number', 'Votes', 'Result', 'name', 'key', 'seats']]
    return df

In [8]:
def create_count_data(url, election):
    """
    read a count file and parse it.
    """
    df = pd.read_csv(url)
    df.rename(columns=count_columns, inplace=True)
    df['key'] = create_keys(df, election)
    df = df[['Result', 'Count Number', 'Non-Transferable', 'Occurred On Count','Transfers',
       'Votes', 'Total Votes', 'Constituency Number', 'Candidate Id', 'key']]
    return df

## That Data Dictionary

Again, the data can quickly get confusing so I thought it best to lock down as a dictionary, as laid out below. It made for more verbose variables, but it was much easier to tell what was what.

```sh
.
├── 2016
│   ├── candidates
│   ├── constituencies
│   └── counts
└── 2020
    ├── candidates
    ├── constituencies
    └── counts
```

In [9]:
data = {2016: {}, 2020: {}}

data[2016]['constituencies'] = create_constituency_data("data/ge2016constituency.csv")
data[2020]['constituencies'] = create_constituency_data("data/ge2020constituency.csv")

data[2016]['candidates'] = create_candidate_data("data/ge2016candidate.csv", "ge16", data[2016]['constituencies'])
data[2020]['candidates'] = create_candidate_data("data/ge2020candidate.csv", "ge20", data[2020]['constituencies'])

data[2016]['count'] = create_count_data("data/ge2016count.csv", "ge16")
data[2020]['count'] = create_count_data("data/ge2020count.csv", "ge20")

## Find First Count Positions

The key process in the entire study.

The find_first_count_positions() function takes three parameters - an id to identify the election in question, a pandas dataframe with all the count details, and a pandas dataframe with all the candidate details.
1. The `first_count_positions` dictionary is created.
2. the data frame with the count details is grouped by Consituency Number.
3. This group is now iterated over. On ever iteration,
4. A new data frame, `temp`, is created by slicing the count data frame to retain first count details only.
5. This new data frame is sorted in descending order by `Total Votes`.
6. A new column is added, `first_count_position`, which is a range running from 1 to the size of data frame plus 1.
7. The `temp` index is changed to the `Candidate Id` column.
8. The `temp` data frame is now iterated over itself.
9. On every iteration, a new key-value pair is added to the `first_count_positions` dictionary. The key is a tuple comprised of the election Id parameter, the Constituency Number and the Candidate Id. The value is the first count position relevant to that particular candidate in that particular election in that particular election.
10. A list, `fcp` (first count position) is created.
11. The candidate data frame is iterated over.
12. The key in the candidate data frame is used to identify the first count position in the `first_count_positions` dictionary, and this value is appended to the list.
13. Finally, the `fcp` list is returned by the function.

In [10]:
def find_first_count_positions(electionId, countDf, candidateDf):
    first_count_positions = {}
    grouper = countDf.groupby('Constituency Number')
    for a, b in grouper:
        temp = b[b['Count Number'] == 1].copy()
        temp.sort_values("Total Votes", ascending=False, inplace=True)
        temp['first_count_position'] = range(1, temp.shape[0]+1)
        temp.index = temp['Candidate Id']
        for i, j in temp.iterrows():
            first_count_positions[(electionId, a, i)] = j.first_count_position

    fcp = []
    for a, b in candidateDf.iterrows():
        fcp.append(first_count_positions[b['key']])
        
    return fcp

In [11]:
first_count_16 = find_first_count_positions("ge16", data[2016]['count'], data[2016]['candidates'])

In [12]:
data[2016]['candidates']['first_count_position'] = first_count_16

In [13]:
first_count_20 = find_first_count_positions("ge20", data[2020]['count'], data[2020]['candidates'])

In [14]:
data[2020]['candidates']['first_count_position'] = first_count_20

## Find Placement - `inside` or `outside`

In [15]:
for i in [2016, 2020]:
    data[i]['candidates']['difference'] = data[i]['candidates']['seats'] - data[i]['candidates']['first_count_position']
    data[i]['candidates']['placement'] = data[i]['candidates']['difference'].apply(lambda x: 'inside' if x >=0 else 'outside')
    data[i]['candidates']['election'] = "ge" + str(i)[2:] 

In [15]:
holder = []
grouper = data[2020]['candidates'].groupby(['Result', 'seats', 'placement'])
for a, b in grouper:
    holder.append({"result": a[0],
                   "seats": a[1],
                   "placement": a[2],
                   "candidates": b.shape[0]})

In [16]:
df2 = pd.DataFrame(holder)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   result      12 non-null     object
 1   seats       12 non-null     int64 
 2   placement   12 non-null     object
 3   candidates  12 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 512.0+ bytes


In [17]:
df2

,result,seats,placement,candidates
0,Elected,3,inside,25
1,Elected,3,outside,5
2,Elected,4,inside,51
3,Elected,4,outside,13
4,Elected,5,inside,62
5,Elected,5,outside,3
6,Not Elected,3,inside,5
7,Not Elected,3,outside,74
8,Not Elected,4,inside,13
9,Not Elected,4,outside,148


In [18]:
pd.pivot_table(df2,
               index=['seats', 'result'],
               columns='placement',
               values='candidates')

placement          inside  outside
seats result                      
3     Elected          25        5
      Not Elected       5       74
4     Elected          51       13
      Not Elected      13      148
5     Elected          62        3
      Not Elected       3      129

In [19]:
holder = []
grouper = data[2020]['candidates'].groupby(['Result', 'seats', 'first_count_position'])
for a, b in grouper:
    holder.append({"result": a[0],
                   "seats": a[1],
                   "first_count_position": a[2],
                   "candidates": b.shape[0]})
df3 = pd.DataFrame(holder)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   result                65 non-null     object
 1   seats                 65 non-null     int64 
 2   first_count_position  65 non-null     int64 
 3   candidates            65 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 2.2+ KB


In [20]:
value_of_first_count_position = pd.pivot_table(df3, index='first_count_position', columns=['result', 'seats'], values='candidates')
value_of_first_count_position.fillna(0, inplace=True)
value_of_first_count_position.astype(int)

result               Elected         Not Elected        
seats                      3   4   5           3   4   5
first_count_position                                    
1                         10  16  13           0   0   0
2                          9  14  13           1   2   0
3                          6  13  13           4   3   0
4                          2   8  11           8   8   2
5                          3   9  12           7   7   1
6                          0   3   1          10  13  12
7                          0   1   2          10  15  11
8                          0   0   0          10  16  13
9                          0   0   0          10  16  13
10                         0   0   0           8  16  13
11                         0   0   0           7  16  13
12                         0   0   0           4  15  13
13                         0   0   0           0  11  13
14                         0   0   0           0   8  10
15                         0   0   0           0   7   8
16                         0   0   0           0   3   4
17                         0   0   0           0   2   2
18                         0   0   0           0   2   2
19                         0   0   0           0   1   1
20                         0   0   0           0   0   1

## All Results

In [21]:
all_candidates =pd.concat([data[2016]['candidates'], data[2020]['candidates']])
all_candidates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1082 entries, 0 to 530
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Constituency          1082 non-null   object
 1   Gender                1082 non-null   object
 2   Party                 1082 non-null   object
 3   Count Number          1082 non-null   int64 
 4   Votes                 1082 non-null   int64 
 5   Result                1082 non-null   object
 6   name                  1082 non-null   object
 7   key                   1082 non-null   object
 8   seats                 1082 non-null   int64 
 9   first_count_position  1082 non-null   int64 
 10  difference            1082 non-null   int64 
 11  placement             1082 non-null   object
 12  election              1082 non-null   object
dtypes: int64(5), object(8)
memory usage: 118.3+ KB


In [22]:
holder = []
grouper = all_candidates.groupby(['Result', 'seats', 'first_count_position'])
for a, b in grouper:
    holder.append({"result": a[0],
                   "seats": a[1],
                   "first_count_position": a[2],
                   "election": b['election'].values[0],
                   "candidates": b.shape[0]})
df4 = pd.DataFrame(holder)
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   result                70 non-null     object
 1   seats                 70 non-null     int64 
 2   first_count_position  70 non-null     int64 
 3   election              70 non-null     object
 4   candidates            70 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 2.9+ KB


In [23]:
value_of_first_count_position_all = pd.pivot_table(df4, index='first_count_position', columns=['result', 'seats'], values='candidates')
value_of_first_count_position.fillna(0, inplace=True)
value_of_first_count_position.astype(int)

result               Elected         Not Elected        
seats                      3   4   5           3   4   5
first_count_position                                    
1                         10  16  13           0   0   0
2                          9  14  13           1   2   0
3                          6  13  13           4   3   0
4                          2   8  11           8   8   2
5                          3   9  12           7   7   1
6                          0   3   1          10  13  12
7                          0   1   2          10  15  11
8                          0   0   0          10  16  13
9                          0   0   0          10  16  13
10                         0   0   0           8  16  13
11                         0   0   0           7  16  13
12                         0   0   0           4  15  13
13                         0   0   0           0  11  13
14                         0   0   0           0   8  10
15                         0   0   0           0   7   8
16                         0   0   0           0   3   4
17                         0   0   0           0   2   2
18                         0   0   0           0   2   2
19                         0   0   0           0   1   1
20                         0   0   0           0   0   1

In [24]:
with open('data.pkl', 'wb') as f: pickle.dump(data, f)